In [290]:
spark

res182: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5db0a86


In [291]:
spark.version

res183: String = 2.4.4


In [292]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

import org.apache.spark.SparkContext
import org.apache.spark.SparkConf


In [293]:
val conf = new SparkConf().setMaster("local")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
val sc = new SparkContext(conf)

conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@4a589163
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4fb691b0


In [294]:
val iterations = 20 // determines the number of iterations
val dataset_path = "D:/BigDataMining/Lab2/data.txt"
val centroids1_path = "D:/BigDataMining/Lab2/c1.txt"
val centroids2_path = "D:/BigDataMining/Lab2/c2.txt"
var euclidean_distance = true

iterations: Int = 20
dataset_path: String = D:/BigDataMining/Lab2/data.txt
centroids1_path: String = D:/BigDataMining/Lab2/c1.txt
centroids2_path: String = D:/BigDataMining/Lab2/c2.txt
euclidean_distance: Boolean = true


In [295]:
// read in files first
// data RDD
val data = sc.textFile(dataset_path)
// centroids RDD
val centroids1 = sc.textFile(centroids1_path)
val centroids2 = sc.textFile(centroids2_path)

data: org.apache.spark.rdd.RDD[String] = D:/BigDataMining/Lab2/data.txt MapPartitionsRDD[1] at textFile at <console>:46
centroids1: org.apache.spark.rdd.RDD[String] = D:/BigDataMining/Lab2/c1.txt MapPartitionsRDD[3] at textFile at <console>:48
centroids2: org.apache.spark.rdd.RDD[String] = D:/BigDataMining/Lab2/c2.txt MapPartitionsRDD[5] at textFile at <console>:49


In [296]:
val dataRdd = data.map(line => line.split(" ").map(_.toFloat))
val centroidsList1 = centroids1.map(line => line.split(" ").map(_.toFloat)).collect()
val centroidsList2 = centroids2.map(line => line.split(" ").map(_.toFloat)).collect()

//var centroids_broadcast = sc.broadcast(centroidsList1)
//centroids_broadcast.destroy()


dataRdd: org.apache.spark.rdd.RDD[Array[Float]] = MapPartitionsRDD[6] at map at <console>:42
centroidsList1: Array[Array[Float]] = Array(Array(0.0, 0.64, 0.64, 0.0, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.64, 0.0, 0.0, 0.0, 0.32, 0.0, 1.29, 1.93, 0.0, 0.96, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.778, 0.0, 0.0, 3.756, 61.0, 278.0, 1.0), Array(0.21, 0.28, 0.5, 0.0, 0.14, 0.28, 0.21, 0.07, 0.0, 0.94, 0.21, 0.79, 0.65, 0.21, 0.14, 0.14, 0.07, 0.28, 3.47, 0.0, 1.59, 0.0, 0.43, 0.43, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.132, 0.0, 0.372, 0.18, 0.048, 5.114, 101.0, 1028.0, 1.0), Array(0.06, 0.0,...

In [297]:
// Assign point p to the cluster with the closest centroid
// and calculate the cost
// return index that indicatse which centroid point belongs and its cost
// ex) (3, 12.345) meaning point belongs to cluster 3 and its cost is 12.345

class k_means extends Serializable {

//def assign_to_centroids(row: Array[Float], centroids: Array[Array[Float]]): 
//    Tuple2[Int, Float] = 
    val euclidean_assign_to_centroids: (Array[Float], org.apache.spark.broadcast.Broadcast[Array[Array[Float]]]) 
    => Tuple2[Int, Float]
    = (row: Array[Float], centroids_broadcast: org.apache.spark.broadcast.Broadcast[Array[Array[Float]]]) => 
    { 
        // calculate min distance first
        // initialize distances between point and centroids with 0's
        var euclidean_distances = Array.fill[Float](10)(0)
        for (centroid_index <- 0 until centroids_broadcast.value.length){
            var euclidean_distance = 0.0
            // calculate Euclidean distance between point and centroid
            for(index <- 0 until row.length){
                // square of (a-b)
                euclidean_distance += scala.math.pow((row(index) - centroids_broadcast.value(centroid_index)(index)), 2)
            }
            // store distance between point and each centroid in eucliean_distances array
            euclidean_distances(centroid_index) = scala.math.sqrt(euclidean_distance).toFloat
        }
        // get the index of min distance, that is clustered point 
        val clustered_point = euclidean_distances.zipWithIndex.minBy(_._1)._2
        // calculate cost function
        val cost = scala.math.pow(euclidean_distances(clustered_point), 2)
        val result = (clustered_point, cost.toFloat)

        // return Tuple2[Int, Float] Int: cluster the point belongs, Float: cost
        result
    }
    
    val manhattan_assign_to_centroids: (Array[Float], org.apache.spark.broadcast.Broadcast[Array[Array[Float]]]) 
    => Tuple2[Int, Float]
    = (row: Array[Float], centroids_broadcast: org.apache.spark.broadcast.Broadcast[Array[Array[Float]]]) => 
    { 
        // calculate min distance first
        // initialize distances between point and centroids with 0's
        var manhattan_distances = Array.fill[Float](10)(0)
        for (centroid_index <- 0 until centroids_broadcast.value.length){
            var manhattan_distance = 0.0
            // calculate Euclidean distance between point and centroid
            for(index <- 0 until row.length){
                // square of (a-b)
                manhattan_distance += scala.math.abs(row(index) - centroids_broadcast.value(centroid_index)(index))
            }
            // store distance between point and each centroid in eucliean_distances array
            manhattan_distances(centroid_index) = manhattan_distance.toFloat
        }
        // get the index of min distance, that is clustered point 
        val clustered_point = manhattan_distances.zipWithIndex.minBy(_._1)._2
        // calculate cost function
        val cost = manhattan_distances(clustered_point)
        val result = (clustered_point, cost.toFloat)

        // return Tuple2[Int, Float] Int: cluster the point belongs, Float: cost
        result
    }
}
/*
val assign_to_centroids: (Array[Float], Array[Array[Float]]) => Tuple2[Int, Float]
    = (row: Array[Float], centroids: Array[Array[Float]]) => 
    { 
        // calculate min distance first
        // initialize distances between point and centroids with 0's
        var euclidean_distances = Array.fill[Float](10)(0)
        for (centroid_index <- 0 until centroids.length){
            var euclidean_distance = 0.0
            // calculate Euclidean distance between point and centroid
            for(index <- 0 until row.length){
                // square of (a-b)
                euclidean_distance += scala.math.pow((row(index) - centroids(centroid_index)(index)), 2)
            }
            // store distance between point and each centroid in eucliean_distances array
            euclidean_distances(centroid_index) = scala.math.sqrt(euclidean_distance).toFloat
        }
        // get the index of min distance, that is clustered point 
        val clustered_point = euclidean_distances.zipWithIndex.minBy(_._1)._2
        // calculate cost function
        val cost = scala.math.pow(euclidean_distances(clustered_point), 2)
        val result = (clustered_point, cost.toFloat)

        // return Tuple2[Int, Float] Int: cluster the point belongs, Float: cost
        result
    }
*/

defined class k_means


In [298]:
// initialize serializable classes
conf.registerKryoClasses(Array(classOf[k_means]))

res184: org.apache.spark.SparkConf = org.apache.spark.SparkConf@4a589163


In [299]:
def kmeans(dataRDD: org.apache.spark.rdd.RDD[Array[Float]], centroids: Array[Array[Float]], iterations: Int, euclidean: Boolean): 
Array[Float] = {
    // euclidean distance
    if (euclidean){
        var centroid_temp = centroids
        // initialize an array to store costs of 20 iterations
        var euclidean_cost = Array.fill[Float](iterations)(0)
        for (iteration <- 0 until iterations){
            // broadcast variable
            var centroids_broadcast = sc.broadcast(centroid_temp)
            // return array of tuple of cluster and cost for each point
            val clusters_costs = dataRDD.map(row => new k_means().euclidean_assign_to_centroids(row, centroids_broadcast)).collect()
            // calculate cost for each iteration
            for (cluster_cost <- clusters_costs){
                euclidean_cost(iteration) += cluster_cost._2 
            }
            // broadcast variable
            val cluster_list = sc.broadcast(clusters_costs)
            // update centroids
            for (centroid_ID <- 0 until centroids.length){
                // row._2.toInt: index of row
                // cluster_list.value(row._2.toInt)._1: closest centroid
                // return (Array[float], long) Long is index
                // filter dataRDD by its corressponding centroid
                val filteredRDD = dataRDD.zipWithIndex.filter(row => cluster_list.value(row._2.toInt)._1 == centroid_ID).collect()
                // calculate mean values
                var means = Array.fill[Float](filteredRDD(0)._1.length)(0)
                for (i <- 0 until filteredRDD(0)._1.length){
                    // find sum of each dimension first
                    for (row <- filteredRDD){
                        means(i) += row._1(i)
                    }
                    means(i) = means(i) / filteredRDD.length
                }
                centroid_temp(centroid_ID) = means
                
            }
            centroids_broadcast.destroy
            cluster_list.destroy
            
        }
        return euclidean_cost
        
    }
    // manhattan distance
    else{
        var centroid_temp = centroids
        var manhattan_cost = Array.fill[Float](iterations)(0)
        for (iteration <- 0 until iterations){
            // broadcast variable
            var centroids_broadcast = sc.broadcast(centroid_temp)
            // return array of tuple of cluster and cost for each point
            val clusters_costs = dataRDD.map(row => new k_means().manhattan_assign_to_centroids(row, centroids_broadcast)).collect()
            // calculate cost for each iteration
            for (cluster_cost <- clusters_costs){
                manhattan_cost(iteration) += cluster_cost._2 
            }
            // broadcast variable
            val cluster_list = sc.broadcast(clusters_costs)
            // update centroids
            for (centroid_ID <- 0 until centroids.length){
                // row._2.toInt: index of row
                // cluster_list.value(row._2.toInt)._1: closest centroid
                // return (Array[float], long) Long is index
                // filter dataRDD by its corressponding centroid
                val filteredRDD = dataRDD.zipWithIndex.filter(row => cluster_list.value(row._2.toInt)._1 == centroid_ID).collect()
                // calculate mean values
                var means = Array.fill[Float](filteredRDD(0)._1.length)(0)
                for (i <- 0 until filteredRDD(0)._1.length){
                    // find sum of each dimension first
                    for (row <- filteredRDD){
                        means(i) += row._1(i)
                    }
                    means(i) = means(i) / filteredRDD.length
                }
                centroid_temp(centroid_ID) = means
                
            }
            centroids_broadcast.destroy
            cluster_list.destroy
            
        }
        return manhattan_cost
        
    }
    
}


kmeans: (dataRDD: org.apache.spark.rdd.RDD[Array[Float]], centroids: Array[Array[Float]], iterations: Int, euclidean: Boolean)Array[Float]


In [300]:
val centroidListTemp = centroidsList1
val result = kmeans(dataRdd, centroidListTemp, 20, true)

println("Max iterations: 20, distance measure: Euclidean")
for (i <- 0 until result.length){
    println("Iteration " + i + " " + result(i))
}

Max iterations: 20, distance measure: Euclidean
Iteration 0 6.2365952E8
Iteration 1 5.09840704E8
Iteration 2 4.85483936E8
Iteration 3 4.63997408E8
Iteration 4 4.60967776E8
Iteration 5 4.60532096E8
Iteration 6 4.60281856E8
Iteration 7 4.59919968E8
Iteration 8 4.59461088E8
Iteration 9 4.5897456E8
Iteration 10 4.5845568E8
Iteration 11 4.57895456E8
Iteration 12 4.5751136E8
Iteration 13 4.57257792E8
Iteration 14 4.5702544E8
Iteration 15 4.5686288E8
Iteration 16 4.56616288E8
Iteration 17 4.56325248E8
Iteration 18 4.56095552E8
Iteration 19 4.55851264E8


centroidListTemp: Array[Array[Float]] = Array(Array(0.16014244, 0.13772246, 0.34355882, 0.050035585, 0.32462642, 0.12060499, 0.0886121, 0.13843422, 0.25113884, 0.35825607, 0.094092526, 0.74548006, 0.1270107, 0.09540925, 0.08295375, 0.2674022, 0.23679712, 0.13565841, 1.477794, 0.12049821, 1.0365839, 0.28900355, 0.14049824, 0.1777936, 0.72797155, 0.16686836, 0.017864764, 0.0139501775, 0.01188612, 0.023167258, 0.0088256225, 0.001494662, 0.095195726, 0.0025622777, 0.01960854, 0.0680427, 0.14195727, 0.0123487525, 0.01797153, 0.02128114, 0.04793594, 0.081316724, 0.012277579, 0.013451958, 0.06135232, 0.14565836, 0.011067616, 0.02316726, 0.075953744, 0.15696086, 0.014373664, 0.3371922, 0.11424556, 0.06230961, 6.1821394, 77.8968, 582.56586, 0.57651246), Array(0.13278267, 0.12191304, 0.27365214, ...

In [301]:
val centroidListTemp = centroidsList1
val result = kmeans(dataRdd, centroidListTemp, 20, false)
println("Max iterations: 20, distance measure: Manhattan")
for (i <- 0 until result.length){
    println("Iteration " + i + " " + result(i))
}

Max iterations: 20, distance measure: Manhattan
Iteration 0 485431.03
Iteration 1 477389.03
Iteration 2 474206.12
Iteration 3 471442.94
Iteration 4 465319.28
Iteration 5 462667.75
Iteration 6 446518.6
Iteration 7 435900.0
Iteration 8 438119.2
Iteration 9 437519.53
Iteration 10 436900.1
Iteration 11 433646.22
Iteration 12 432701.44
Iteration 13 430970.56
Iteration 14 429909.62
Iteration 15 429331.66
Iteration 16 430681.47
Iteration 17 431801.6
Iteration 18 432634.72
Iteration 19 433334.12


centroidListTemp: Array[Array[Float]] = Array(Array(0.1374611, 0.1427461, 0.35186535, 0.1734197, 0.2809844, 0.12963729, 0.07512952, 0.15279792, 0.39518145, 0.46072537, 0.10227984, 0.5068396, 0.13818651, 0.08943004, 0.14678754, 0.25823832, 0.123626955, 0.3592747, 1.4841967, 0.22533676, 0.9222798, 0.8212435, 0.18673581, 0.1927461, 0.3910881, 0.15067355, 0.020103624, 0.020414505, 0.0060103633, 0.018082902, 0.022746116, 0.0014507772, 0.09471503, 0.0031088083, 0.10974094, 0.029430056, 0.14005181, 0.001554404, 0.021658033, 0.030414505, 0.008445595, 0.027098443, 0.0070984457, 0.011554404, 0.04041451, 0.039792743, 0.009430052, 0.030051813, 0.047378235, 0.12311915, 0.042321235, 0.30423307, 0.17210877, 0.09938342, 5.8380957, 108.17616, 1105.43, 0.6683938), Array(0.29879996, 0.14300004, 0.28139997...

In [302]:
val centroidListTemp = centroidsList2
val result = kmeans(dataRdd, centroidListTemp, 20, true)
println("Max iterations: 20, distance measure: Euclidean")
for (i <- 0 until result.length){
    println("Iteration " + i + " " + result(i))
}

Max iterations: 20, distance measure: Euclidean
Iteration 0 4.38748608E8
Iteration 1 2.4980408E8
Iteration 2 1.94494464E8
Iteration 3 1.69804416E8
Iteration 4 1.5629504E8
Iteration 5 1.49094368E8
Iteration 6 1.42507952E8
Iteration 7 1.32303768E8
Iteration 8 1.17170712E8
Iteration 9 1.08547072E8
Iteration 10 1.022374E8
Iteration 11 9.8278256E7
Iteration 12 9.5630008E7
Iteration 13 9.3793288E7
Iteration 14 9.2376576E7
Iteration 15 9.15418E7
Iteration 16 9.10456E7
Iteration 17 9.075248E7
Iteration 18 9.0470064E7
Iteration 19 9.0216304E7


centroidListTemp: Array[Array[Float]] = Array(Array(0.08928571, 0.23088643, 0.24967588, 0.022641996, 0.3007745, 0.079850845, 0.10706823, 0.089368895, 0.034584057, 0.19471608, 0.04306656, 0.51638836, 0.0800487, 0.03851693, 0.023235805, 0.24027808, 0.11290593, 0.16526401, 1.6708814, 0.048126787, 0.73868865, 0.023582906, 0.07014054, 0.07275675, 0.57496274, 0.30552495, 0.99994266, 0.15644294, 0.124979876, 0.12755018, 0.078158334, 0.060275376, 0.103063665, 0.060923684, 0.1246328, 0.11281409, 0.13727768, 0.014047621, 0.096362576, 0.07073435, 0.05026678, 0.16089214, 0.053895593, 0.0990591, 0.3745525, 0.20306948, 0.004520941, 0.034239817, 0.021968156, 0.13711667, 0.016120192, 0.23903295, 0.049374316, 0.03197792, 2.4822683, 17.213139, 80.87808, 0.3118187), Array(0.19, 0.0, 0.38, 0.0, 0.0, 0.19, ...

In [303]:
val centroidListTemp = centroidsList2
val result = kmeans(dataRdd, centroidListTemp, 20, false)
println("Max iterations: 20, distance measure: Manhattan")
for (i <- 0 until result.length){
    println("Iteration " + i + " " + result(i))
}

Max iterations: 20, distance measure: Manhattan
Iteration 0 614313.9
Iteration 1 617247.2
Iteration 2 619734.94
Iteration 3 621999.56
Iteration 4 623836.4
Iteration 5 624444.2
Iteration 6 625149.5
Iteration 7 625356.3
Iteration 8 625464.56
Iteration 9 625568.3
Iteration 10 625568.3
Iteration 11 625568.3
Iteration 12 625568.3
Iteration 13 625568.3
Iteration 14 625568.3
Iteration 15 625568.3
Iteration 16 625568.3
Iteration 17 625568.3
Iteration 18 625568.3
Iteration 19 625568.3


centroidListTemp: Array[Array[Float]] = Array(Array(0.090335205, 0.22750147, 0.24936059, 0.022670424, 0.30014074, 0.08075212, 0.10594081, 0.08902253, 0.03467606, 0.19277471, 0.043084525, 0.51696044, 0.08104501, 0.03856902, 0.022842258, 0.23873222, 0.1137606, 0.16507336, 1.6661725, 0.048839435, 0.7350393, 0.029304227, 0.071752094, 0.07269297, 0.5797915, 0.3048001, 0.98374933, 0.15469015, 0.123952076, 0.12662536, 0.07736899, 0.05941689, 0.10311547, 0.06005351, 0.12336055, 0.11270422, 0.13913241, 0.0138478875, 0.09491267, 0.069732375, 0.05010422, 0.15883663, 0.053735223, 0.09775495, 0.36968732, 0.20621416, 0.004439437, 0.03397747, 0.023063663, 0.13769484, 0.016350701, 0.23781514, 0.049758274, 0.03230198, 2.4908764, 17.37324, 84.97606, 0.3140845), Array(0.19, 0.0, 0.38, 0.0, 0.0, 0.19, 0.0,...

In [289]:
sc.stop()